<a href="https://colab.research.google.com/github/chadory13/cnn_chart_pattern_-filter/blob/main/cup%26handle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models


import numpy as np
import cv2

def clip_to_image_size(boxes, image_height, image_width):
    """
    boxes: Tensor or ndarray of shape (N, 4), each box = [x1, y1, x2, y2]
    image_height, image_width: int
    """
    boxes[:, 0] = np.clip(boxes[:, 0], 0, image_width)    # x1
    boxes[:, 1] = np.clip(boxes[:, 1], 0, image_height)   # y1
    boxes[:, 2] = np.clip(boxes[:, 2], 0, image_width)    # x2
    boxes[:, 3] = np.clip(boxes[:, 3], 0, image_height)   # y2
    return boxes

# 데이터 증강 및 로딩
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=8,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

# Google Drive 경로 설정
google_drive_path = r'/content/drive/MyDrive/Colab Notebooks/cup'

train_gen = datagen.flow_from_directory(
    google_drive_path,
    target_size=(64,64),
    batch_size=8,
    class_mode='categorical',    # 다중 클래스 분류는 'categorical'
    subset='training'
)

val_gen = datagen.flow_from_directory(
    google_drive_path,
    target_size=(64,64),
    batch_size=8,
    class_mode='categorical',
    subset='validation'
)

# EfficientNetB0를 사전 학습 가중치로 불러오고, 최상위 분류층은 직접 붙임
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(64,64,3))
base_model.trainable = False

num_classes = train_gen.num_classes

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
model.fit(train_gen, epochs=10, validation_data=val_gen)

# 학습 직후에만 필요한 게 아니라,
# 예측을 하는 모든 셀에서 inv_class_indices가 필요합니다.
class_indices     = train_gen.class_indices
inv_class_indices = {v:k for k,v in class_indices.items()}


# 1) 예측할 이미지 경로 리스트 정의
test_image_paths = [
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test1.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test2.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test3.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test4.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test5.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test6.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test7.png',
    '/content/drive/MyDrive/Colab Notebooks/cup/test.png/test8.png'

]

# ] 2) for문으로 리스트 순회
for path in test_image_paths:
    img = cv2.imread(path)
    # ] 3) 파일 존재 여부 체크
    if img is None:
        print(f"이미지 없음: {path}")
        continue

    img = cv2.resize(img, (64,64)) / 255.0
    pred = model.predict(np.expand_dims(img, axis=0))
    pred_class_idx = np.argmax(pred[0])

    # [변경] 4) 출력 메시지에 파일명도 함께 표시
    print(f"{path} → Predicted class: {inv_class_indices[pred_class_idx]}")

Found 167 images belonging to 6 classes.
Found 41 images belonging to 6 classes.
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 282ms/step - accuracy: 0.2101 - loss: 1.6993 - val_accuracy: 0.2439 - val_loss: 1.5042
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.2316 - loss: 1.5681 - val_accuracy: 0.2439 - val_loss: 1.4795
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.3014 - loss: 1.5058 - val_accuracy: 0.2439 - val_loss: 1.4887
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.2029 - loss: 1.6015 - val_accuracy: 0.2439 - val_loss: 1.5175
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.1929 - loss: 1.5529 - val_accuracy: 0.2439 - val_loss: 1.4813
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.2902 - loss: 1.4752 - val_accuracy: 0.2439 - val_loss: 1.5049
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.2357 - loss: 1.5898 - val_accuracy: 0.2439 - val_loss: 1.5180
Epoch 8/10
21/21 ━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
